In [93]:
#import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import config
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [3]:
#sqlalcehmy database connection
database = f'postgres://{config.user}:{config.db_password}@{config.host}/postgres'
engine = create_engine(database)

In [4]:
beer_db = pd.read_sql_query('''SELECT * FROM beer_data;''', con=engine)
beer_db.head()

,abv,ibu,id,name,style,brewery_id,ounces,brewery_idbrewery_,namebrewery_,city,state
0,0.061,60.0,1979,Bitter Bitch,American Pale Ale (APA),177,12.0,177,18th Street Brewery,Gary,IN
1,0.099,92.0,1036,Lower De Boom,American Barleywine,368,8.4,368,21st Amendment Brewery,San Francisco,CA
2,0.079,45.0,1024,Fireside Chat,Winter Warmer,368,12.0,368,21st Amendment Brewery,San Francisco,CA
3,0.044,42.0,876,Bitter American,American Pale Ale (APA),368,12.0,368,21st Amendment Brewery,San Francisco,CA
4,0.049,17.0,802,Hell or High Watermelon Wheat (2009),Fruit / Vegetable Beer,368,12.0,368,21st Amendment Brewery,San Francisco,CA


In [80]:
# splitting the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(beer_db[['ibu','abv']], beer_db[['style']], random_state=48)

# Creating scaler instance

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train

,style
1103,Bock
80,Gose
414,American IPA
844,American Stout
612,American Strong Ale
...,...
966,American Pale Ale (APA)
944,Hefeweizen
347,American IPA
1361,American IPA


In [46]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
fvals = le.fit_transform(np.ravel(beer_db[['style']]))
fvals = np.unique(fvals)
print(fvals)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89]


In [104]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, random_state=48).fit(X_train,(np.ravel(y_train)), sample_weight=None)

print(rfc)

y_preds = rfc.predict(X_test)

RandomForestClassifier(random_state=48)


In [105]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_preds))
print(classification_report(y_test,y_preds))
print(accuracy_score(y_test, y_preds))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 2 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]
                                     precision    recall  f1-score   support

                   Abbey Single Ale       0.00      0.00      0.00         0
                            Altbier       0.00      0.00      0.00         3
             American Adjunct Lager       1.00      0.50      0.67         4
           American Amber / Red Ale       0.44      0.35      0.39        20
         American Amber / Red Lager       0.00      0.00      0.00         4
                American Barleywine       0.00      0.00      0.00         0
                 American Black Ale       0.20      0.25      0.22         4
                American Blonde Ale       0.24      0.29      0.26        17
                 American Brown Ale       0.00      0.00      0.00         8
            American Dark Wheat Ale       0.50      0.50      0.50         2
     American Double / Imperial 

C:\Users\chrer\anaconda3\envs\PythonData\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chrer\anaconda3\envs\PythonData\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
